In [13]:
# need to filter first through simple string search and manual logic; then apply fuzzy matching to filter the leftovers
# like in notebook Fuzz

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
from csv import reader
from fuzzywuzzy import fuzz, process

stopwords = set(stopwords.words("english"))

dictionary = pd.read_csv('E:\JS\Py\Text_similarity\Fuzzy.csv', header=None, squeeze=True).to_dict()
#sec_dict1 = pd.read_csv('E:\JS\Py\Text_similarity\Second_dict.csv', header=None, squeeze=True).to_dict()
with open('E:\JS\Py\Text_similarity\Second_dict.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    sec_dict = list(map(tuple, csv_reader))

master_doc = []
match_doc = []

def stringsearch(string, sub_str, sub_str1, line):
    if(string.find(sub_str)!=-1 and string.find(sub_str1)!=-1):
        final_doc.append(sub_str+" "+sub_str1)
        clean_doc.remove(line)
        return (1)
        
def secondsearch(string, sub_str, line):
    r = fuzz.token_set_ratio(string, sub_str)
    #r = fuzz.token_sort_ratio(string, sub_str)
    if(r > 90):
        final_doc.append(sub_str)
        clean_doc.remove(line)
        return (1)

def listtostring(clean_line):
    return(" ".join(clean_line))

def replacer(string1):
    return(string1.replace('\bscr\b','screw').replace('\bspr\b','spring').replace('\bbd\b','band').replace('spr bd','spring band')
           .replace('\bw/h\b','wiring harness').replace('brg','bearing').replace('bkt','bracket').replace('\bwh\b','wiring harness')
           .replace('w/h','wiring harness').replace('\breinf\b','reinforcement').replace('\bwh\b','wiring harness').replace('mtg.','mounting')
           .replace('\bmtg\b','mounting').replace('\bbrkt\b','bracket').replace('\brr\b','rear').replace('frt','front').replace('\bdr\b','door')
           .replace('reinfrocement','reinforcement').replace('mountingbracket','mounting bracket').replace('pipe bunch','nylon bunch').replace('vent tube','vent hose').replace('\bvent\b','ventilation')
           .replace('suction tube','suction hose').replace('\bcyl\b','cylinder').replace('suction pipe','suction hose').replace('suction line','suction hose').replace('fuel return pipe','fuel return hose')
           .replace('coolant pipe','coolant hose').replace('exh','exhaust').replace('batt','battery').replace('loadbody','load body').replace('\barb\b','anti roll bar').replace('window regulator','window winding')
           .replace('pressure line','pressure hose').replace('return line','return hose').replace('gear box','gearbox'))
   
#using Lemmatization
lemma = nltk.wordnet.WordNetLemmatizer()

file_docs = []
test_doc = []
test_line = []

with open('E:\JS\Py\Text_similarity\Doc5.csv', 'r', encoding="UTF-8") as inp, open('E:\JS\Py\Text_similarity\Doc3.txt', 'w') as out:
    for line in inp:
        out.write(line + '.\n')

with open ('E:\JS\Py\Text_similarity\Doc3.txt') as f:
    tokens = sent_tokenize(f.read())
    #wtokens = word_tokenize(f.read())
    for line in tokens:
        file_docs.append(line)
        

# create a clean document after removing the stop-words and generate lemmas
clean_doc = []

for line in file_docs:
    clean_line = [word for word in line.split() if word not in stopwords]
    
    for word in line.split():
        test_line.append(lemma.lemmatize(word))
    test_doc.append(listtostring(test_line))
    test_line = []
    
    #clean_doc.append(listtostring(clean_line))
    clean_doc = test_doc

final_doc=[]
init_size = len(clean_doc)

##first loop of search and filtering
# for line in clean_doc:
    
#     line1 = line.lower()
#     string = line1.partition(",")[0].partition(";")[0].partition("(")[0].partition("[")[0].partition('for')[0].partition('with')[0].partition('w/o')[0].partition('offer')[0].partition('\bw\b')[0]

#     for y in sec_dict:
#         if (stringsearch(replacer(string.lower()),y[0].lower(),y[1].lower(), line) == 1):
#             print(line, " is ::-->")
#             break
#     if(len(final_doc)!=0):
#         print(final_doc)
#     final_doc = [] 
    

##second loop of search and filtering (fuzzy search)
for line in clean_doc:
    
    line1 = line.lower()
    string = line1.partition(",")[0].partition(";")[0].partition("(")[0].partition("[")[0].partition('for')[0].partition('with')[0].partition('w/o')[0].partition('offer')[0].partition('\bw\b')[0]

    closest_match = process.extractOne(replacer(string).lower(), dictionary, scorer=fuzz.token_set_ratio)
    
    if(closest_match[1] >= 90):
        print(line, " is ::-->", closest_match)
        clean_doc.remove(line)
    
#     for x in dictionary:
        
#         if (secondsearch(replacer(string.lower()),dictionary[x].lower(), line) == 1):
#             print(line, " is ::-->")
#             break
            
#     if(len(final_doc)!=0):
#         print(final_doc)
#     final_doc = [] 
    
print(len(clean_doc))

SUCTION LINE ASSY .  is ::--> ('Suction Hose', 100, 35)
RETURN LINE ASSY .  is ::--> ('Return Hose', 100, 42)
"WIRING HARNESS,AEBS FRAME (52WB) (282154621601)" .  is ::--> ('Floor Wiring Harness', 100, 19)
"DRUM BRAKE ASSEMBLY WITH DC, RR LH (140mm)" .  is ::--> ('Drum  Brake', 100, 230)
FAN SHROUD ASSY .  is ::--> ('Fan Shroud', 100, 113)
BRACKET .  is ::--> ('Bracket ', 100, 5)
"L-BRACKET,EXHAUST PIPE MTG" .  is ::--> ('Bracket ', 100, 5)
BRACKET ASSY CABLE ANCHORING SHIFT .  is ::--> ('Bracket ', 100, 5)
GEAR SHIFTER ASSYÂ (M/s Excel) .  is ::--> ('Gear Shifter ', 100, 250)
WHEEL RIM ASSY WITH VALVE (205 PCD) .  is ::--> ('Wheel Rim ', 100, 71)
"PIPE ASSY,JOINT TO JOINT (29WB)" .  is ::--> ('Pipe Assy ', 100, 100)
Holder Parking Sensor .  is ::--> ('Parking Sensor', 100, 159)
"PLATE,SPARE WHEEL MTG" .  is ::--> ('Spacer Plate', 100, 78)
CLAMP PIPE .  is ::--> ('Clamp ', 100, 8)
PROPELLER SHAFT ASSY DRIVE .  is ::--> ('Drive Shaft', 100, 217)
PIPE BUNCH ASSY .  is ::--> ('Nylon Bunch

FUEL FILTER ASSY .  is ::--> ('Fuel Filter', 100, 295)
SNORKEL ASSY .  is ::--> ('Snorkel ', 100, 292)
"BRACKET ASSY,CLUTCH & BRAKE PEDAL MT" .  is ::--> ('Bracket ', 100, 5)
"OUTER PANEL,FRONT DOOR,LH" .  is ::--> ('Panel ', 100, 212)
"CLRPRT_ASY_1-DOOR TRIM ASSY,FRT DR M" .  is ::--> ('Door Trim', 100, 302)
CROSS MEMBER ASSY .  is ::--> ('Cross Member ', 100, 72)
"SHOCK ABSORBER ASSY,REAR" .  is ::--> ('Shock Absorber ', 100, 138)
"FRONT PANEL,OUTER" .  is ::--> ('Panel ', 100, 212)
"RETURN LINE ASSY,GEAR BOX TO RESERVO" .  is ::--> ('Return Hose', 100, 42)
"B-PILLAR ASSY,INNER-LH" .  is ::--> ('B Pillar', 100, 315)
PARKING BRAKE LEVER ASSY .  is ::--> ('Parking Brake Lever', 100, 190)
FUEL RETURN PIPE ASSY .  is ::--> ('Return Hose', 100, 42)
"SHOCK ABSORBER ASSY,FRONT" .  is ::--> ('Shock Absorber ', 100, 138)
"COVER ASSY,FRONT WHEEL ARCH,LH" .  is ::--> ('Cover ', 100, 207)
"ACCESS COVER,RADIATOR" .  is ::--> ('Cover ', 100, 207)
BRACKET ASSY .  is ::--> ('Bracket ', 100, 5)
BRACK

CABLE ASSY STARTER MOTOR +VE .  is ::--> ('Cable ', 100, 228)
CLAMP .  is ::--> ('Spring Band Clamp ', 100, 7)
ECU .  is ::--> ('EMS ECU', 100, 347)
BRACKET .  is ::--> ('Bracket ', 100, 5)
PIPE ASSY .  is ::--> ('Pipe Assy ', 100, 100)
BRACKET STG GEAR BOX MTG .  is ::--> ('Bracket ', 100, 5)
BALATA PACKING .  is ::--> ('Balata ', 100, 134)
COBRA BRACKET ASSY .  is ::--> ('Bracket ', 100, 5)
BRACKET ASSY FRONT SPRING HANGER REAR .  is ::--> ('Bracket ', 100, 5)
BRACKET SPRING HANGER FRONT-FRONT .  is ::--> ('Bracket ', 100, 5)
TUBELESS TYRE .  is ::--> ('Tubeless Tyre', 100, 245)
CLUTCH HOSE ASSY .  is ::--> ('Clutch Hose', 100, 39)
VALVE EXTENSION .  is ::--> ('Valve ', 100, 187)
LAMP STOP/TAIL - 12V .  is ::--> ('Tail Lamp', 100, 262)
CLUTCH RELEASE YOKE .  is ::--> ('Yoke ', 100, 248)
RUBBER BUFFER FRONT AXLE .  is ::--> ('Front Axle', 100, 342)
ASSY.  is ::--> ('Frame Assy ', 100, 76)
THREADED PIN .  is ::--> ('Pin ', 100, 345)
PIPE .  is ::--> ('Air Intake Pipe', 100, 90)
HOSETHE

Accelerator pedal assembly (electronic) .  is ::--> ('Accelerator Pedal', 100, 198)
Front RH and LH leaf spring mounting U bolt .  is ::--> ('Mounting Bolt', 100, 0)
Dashboard skin .  is ::--> ('Dashboard ', 100, 283)
ORVM RH .  is ::--> ('ORVM ', 100, 323)
Group No 91_Front Seat (Driver Seat) .  is ::--> ('Seat ', 100, 333)
FUEL PIPE ASSY TANK TO FILTER .  is ::--> ('Pipe Assy ', 100, 100)
"Wiring Harness, ABS" .  is ::--> ('Floor Wiring Harness', 100, 19)
"Wiring Harness, ABS" .  is ::--> ('Floor Wiring Harness', 100, 19)
Vent hose (RADIATOR TO AUX.  is ::--> ('Hose ', 100, 49)
"PIPE ASSY,ENGINE TO AUX WATER TANK" .  is ::--> ('Pipe Assy ', 100, 100)
SUPPORT BRACKET ASSY .  is ::--> ('Bracket ', 100, 5)
SUPPORT BRACKET ASSY .  is ::--> ('Bracket ', 100, 5)
Assy Intercooler pipe .  is ::--> ('Pipe Assy ', 100, 100)
BRACKET ASSY ENGINE MTG BOTTOM RH .  is ::--> ('Bracket ', 100, 5)
GEAR SHIFTER ASSY .  is ::--> ('Gear Shifter ', 100, 250)
WHEEL RIM ASSY WITH VALVE .  is ::--> ('Wheel R

ASSY SPACER[FOR EMER RECEPTACLE MTG] .  is ::--> ('Spacer ', 100, 16)
MALE CONNECTOR 1/4 NPT (M/S SWAGELOK) .  is ::--> ('Connector ', 100, 108)
ASSY RECEPTACLE CAP (M/S MINDA) .  is ::--> ('Receptacle ', 100, 293)
HP TUBE ASSY .  is ::--> ('HP Tube ', 100, 84)
VENTILATION HOSE (M/S.MINDA) .  is ::--> ('Ventilation Hose', 100, 31)
ASSY CYLINDER +CYL VALVE(MOON) .  is ::--> ('Cylinder ', 100, 115)
VENTILATION HOSE (M/S.MINDA) .  is ::--> ('Ventilation Hose', 100, 31)
HP TUBE ASSY .  is ::--> ('HP Tube ', 100, 84)
SLEEVE (FOR CNG PIPE) .  is ::--> ('Sleeve ', 100, 157)
HP TUBE ASSY .  is ::--> ('HP Tube ', 100, 84)
MALE CONNECTOR 1/4 NPT (M/S SWAGELOK) .  is ::--> ('Connector ', 100, 108)
LP HOSE ASSY HP REGULATOR TO LP FILTER .  is ::--> ('LP Hose', 100, 44)
CYLINDER CRANKCASE ASSY WITH LINER .  is ::--> ('Cylinder ', 100, 115)
VALVE SEAT INTAKE .  is ::--> ('Valve Seat ', 100, 57)
VALVE SEAT INTAKE TC .  is ::--> ('Valve Seat ', 100, 57)
VALVE SEAT EXHAUST .  is ::--> ('Valve Seat ', 1

...

DRUM BRAKE ASSY WITH DC;REAR-LH .  is ::--> ('Drum  Brake', 100, 230)
DRUM BRAKE ASSY WITH DC;REAR-LH .  is ::--> ('Drum  Brake', 100, 230)
BRACKET .  is ::--> ('Bracket ', 100, 5)
LONG MEMBER;LH .  is ::--> ('Long Member ', 100, 73)
BRACKET ASSY;RESERVOIR MTG .  is ::--> ('Bracket ', 100, 5)
HIGH PRESSURE LINE ASSY;GB TO P/CYL .  is ::--> ('Hose ', 100, 49)
FUEL TANK ASSY W/O TANK UNIT .  is ::--> ('Fuel Tank', 100, 267)
PROPELLER SHAFT ASSY;DRIVE .  is ::--> ('Propeller Shaft', 100, 220)
SWITCH;HILL ASSIST .  is ::--> ('Switch Bezel ', 100, 60)
EXHAUST PIPE ASSY WITH FLEX BELLOW .  is ::--> ('Pipe Assy ', 100, 100)
WIRING HARNESS;ABS FRAME .  is ::--> ('Floor Wiring Harness', 100, 19)
WIRING HARNESS;TAIL .  is ::--> ('Floor Wiring Harness', 100, 19)
METAL PIPE ASSY WITH CRIMPED TEFLON HOSE .  is ::--> ('Metal Pipe', 100, 95)
WIRING HARNESS;TAIL .  is ::--> ('Floor Wiring Harness', 100, 19)
WIRING HARNESS;ABS FRAME .  is ::--> ('Floor Wiring Harness', 100, 19)


CLUTCH BOOSTER ASSY .  is ::--> ('Clutch Booster', 100, 120)
DROP GEAR HOUSING .  is ::--> ('Gear ', 100, 252)
"WIRING HARNESS,ABS FRAME" .  is ::--> ('Floor Wiring Harness', 100, 19)
SPRING BRAKE ACTUATOR .  is ::--> ('Brake ', 100, 234)
WIRING HARNESS;CABIN .  is ::--> ('Floor Wiring Harness', 100, 19)
WIRING HARNESS;EMS .  is ::--> ('Floor Wiring Harness', 100, 19)
WIRING HARNESS;BATTERY CABLE +VE .  is ::--> ('Floor Wiring Harness', 100, 19)
WIRING HARNESS;CHASSIS .  is ::--> ('Floor Wiring Harness', 100, 19)
BRACKET ASSY.  is ::--> ('Bracket ', 100, 5)
METAL PIPE ASSY .  is ::--> ('Metal Pipe', 100, 95)
BRACKET ASSY .  is ::--> ('Bracket ', 100, 5)
NYLON PIPE BUNCH ASSY .  is ::--> ('Nylon Bunch', 100, 83)
SPRING PIN .  is ::--> ('Pin ', 100, 345)
LP HOSE ASSY .  is ::--> ('LP Hose', 100, 44)


FUPD Assy .  is ::--> ('FUPD ', 100, 325)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Frame Assy .  is ::--> ('Frame Assy ', 100, 76)
Air Filter with adaptor (279909136304)- complete M/s Mahle .  is ::--> ('Air Filter', 100, 296)
LCL hose Yodha .  is ::--> ('Hose ', 100, 49)


bracket top LH .  is ::--> ('Bracket ', 100, 5)
bracket Top RH .  is ::--> ('Bracket ', 100, 5)
bracket Bottom LH .  is ::--> ('Bracket ', 100, 5)
bracket Bottom RH .  is ::--> ('Bracket ', 100, 5)
Brackets for Lambda sensor connector mtg.  is ::--> ('Bracket ', 93, 5)
BRACKET Alternator/PS pump mtg.  is ::--> ('Bracket ', 100, 5)
Brake Electrical Valves - Front .  is ::--> ('Brake ', 100, 234)
Brake Electrical Valves - Front .  is ::--> ('Brake ', 100, 234)
Service Air Tank .  is ::--> ('Air Tank', 100, 266)
Additional Air Tank .  is ::--> ('Air Tank', 100, 266)
NYLON PIPE BUNCH ASSY .  is ::--> ('Nylon Bunch', 100, 83)
ASSY GB HSG FRT+INTER+REAR DD Neutral Switch .  is ::--> ('Switch ', 100, 156)
GEAR BOX HOUSING INTERMEDIATE G1150 .  is ::--> ('Gearbox ', 100, 251)
PIPE ASSY .  is ::--> ('Pipe Assy ', 100, 100)
METAL PIPE ASSY .  is ::--> ('Metal Pipe', 100, 95)
PIPE ASSY .  is ::--> ('Pipe Assy ', 100, 100)
"LONG MEMBER,RH" .  is ::--> ('Long Member ', 100, 73)
"PROPELLER SHAFT ASS

"PIPE ASSY,INTERCOOLER TO INTAKE THROTTLE" .  is ::--> ('Pipe Assy ', 100, 100)
"HOSE,INTERCOOLER" .  is ::--> ('Heater Hose', 100, 30)
"RUBBER HOSE,PIPE TO INTERCOOLER" .  is ::--> ('Rubber Hose', 100, 33)
EXHAUST PIPE ASSY WITH FLEX BELLOW .  is ::--> ('Pipe Assy ', 100, 100)
EXHAUST PIPE ASSY WITH FLEX BELLOW .  is ::--> ('Pipe Assy ', 100, 100)
Fwd axle assy-Otimized 37T bogie .  is ::--> ('Axle ', 100, 344)
Rear spring bush .  is ::--> ('Bush ', 100, 240)
"U-CLAMP,REAR" .  is ::--> ('Clamp ', 100, 8)
L BRACKET WIRING HARNESS MTG .  is ::--> ('Bracket ', 100, 5)
L BRACKET WIRING HARNESS MTG .  is ::--> ('Bracket ', 100, 5)
ASSY DASHBOARD RHD EBONY BLACK AIRBAG .  is ::--> ('Dashboard ', 100, 283)
NACELLE TOP RHD D380 COLLAPS STG FOR .  is ::--> ('Nacelle ', 100, 346)
Dummy Axle Beam .  is ::--> ('Beam ', 100, 260)
BRAKE DRUM .  is ::--> ('Drum  Brake', 100, 230)
"SUPPORT BRACKET, AIR INTAKE PIPE" .  is ::--> ('Bracket ', 100, 5)
"SPINDLE (ACBB), REAR TWIST BEAM" .  is ::--> ('Spind

In [20]:
# def thirdsearch(string, sub_str, line):
#     r = fuzz.token_sort_ratio(string, sub_str)
#     if(r > 80):
#         final_doc.append(sub_str)
#         clean_doc.remove(line)
#         return (1)

final_doc=[]
init_size = len(clean_doc)
for line in clean_doc:
    
    line1 = line.lower()
    string = line1.partition(",")[0].partition(";")[0].partition("(")[0].partition("[")[0].partition('for')[0].partition('with')[0].partition('w/o')[0].partition('offer')[0].partition('\bw\b')[0]

    for x in dictionary:
        
        if (secondsearch(replacer(string.lower()),dictionary[x].lower(), line) == 1):
            print(line, " is ::-->")
            break
    if(len(final_doc)!=0):
        print(final_doc)
    final_doc = [] 
    
print(len(clean_doc))

"WIRING HARNESS,AEBS FRAME (45WB) (264254621603)" .  is ::-->
['floor wiring harness']
PIPE BUNCH ASSY (33WB) .  is ::-->
['nylon bunch']
PLUG (STUB AXLE-TOP) .  is ::-->
['drain plug ']
SPACER RUBBER .  is ::-->
['rubber spacer']
BRACKET MUDGUARD MTG RH .  is ::-->
['bracket ']
"LEAF SPRING ASSY,FRONT-RH" .  is ::-->
['leaf spring ']
FRONT SEAT ASSY RH 2W .  is ::-->
['front seat']
"COVER ASSY,CLUSTER" .  is ::-->
['cover ']
Rear Door Rub Rail LH .  is ::-->
['door ']
SWITCH EXHAUST BRAKE FOR 3L ENGINE .  is ::-->
['switch ']
BRAKE CHAMBER RH .  is ::-->
['brake ']
"METAL PIPE ASSY, JOINT 1 TO JOINT 2" .  is ::-->
['metal pipe']
FUEL TANK ASSY 60L .  is ::-->
['fuel tank']
METAL PIPE ASSY INT JOINT TO DDU .  is ::-->
['metal pipe']
SUCTION PIPE ASSY PRE FILTER TO FILTER .  is ::-->
['suction  hose']
CROSS TUBE ASSY .  is ::-->
['tube assy ']
BUNDY TUBE ASSY CLUTCH PIPE .  is ::-->
['bundy tube']
BOTTOM BKT (7.5T) .  is ::-->
['bracket ']
WIRING HARNESS FIREWALL .  is ::-->
['wiring ha

TELEMATICS ECU .  is ::-->
['ecu ']
"CABLE,PDU TO DC-DC CONVERTER" .  is ::-->
['cable tie ']
"CABLE,HV GND AC COMPRESSOR TO CHASSISS GND" .  is ::-->
['cable tie ']
"W/H, FRONT LIMBOR XM" .  is ::-->
['floor wiring harness']
"W/H, COCKPIT LIMBER XM, XM(O)" .  is ::-->
['floor wiring harness']
"W/H, FLOOR LH LIMBER XM, XM(O)" .  is ::-->
['floor wiring harness']
"LONG MEMBER EXTENSION,FRONT-LH" .  is ::-->
['long member ']
DRIVE SHAFT ASSY RH .  is ::-->
['drive  shaft']
SOLENOID VALVE ASSY .  is ::-->
['solenoid  valve']
INSULATION FRONT END FLOOR .  is ::-->
['insulation ']
FRONT BUMPER ASSY .  is ::-->
['front bumper']
Instrument Cluster Digital (542416262019) .  is ::-->
['instrument cluster']
AUX WATER TANK ASSY WITH PRESSURE CAP .  is ::-->
['aux tank']
TUBE ASSY hose to rear Wc RH .  is ::-->
['hose ']
"Front seat assy RH, 4W, J2 structure (XZ+)" .  is ::-->
['front seat']
"Rear seat assy, Armrest with 2 Cup holder, 100% Fixed (XZ+)" .  is ::-->
['rear seat']
Intake Valve .  is 

Engine cradle rear LH and RH mounting washer .  is ::-->
['washer ']
Front damper upper mounting nut LH and RH .  is ::-->
['mounting nut ']
Front LH and RH damper upper mounting washer 1 .  is ::-->
['front damper']
Front RH and LH leaf spring mounting plane washer .  is ::-->
['spring washer ']
Front leaf spring to shackle mounting nut .  is ::-->
['mounting nut ']
Leaf spring shackle plate to leaf spring mounting nut (LH & RH) .  is ::-->
['mounting nut ']
Leaf spring front end plate with stud mounting nut .  is ::-->
['plate ']
Front knuckle assembly LH and RH upper mounting lock nut .  is ::-->
['mounting nut ']
Wheel hub bearing oil seal 1 .  is ::-->
['oil seal ']
Wheel hub bearing circlip .  is ::-->
['bearing ']
Spare wheel carriage pivot washer .  is ::-->
['washer ']
Spare wheel winch mounting washer .  is ::-->
['washer ']
Front Wheel mounting bolt (LH & RH) .  is ::-->
['mounting bolt']
Front brake hose holding bracket RH .  is ::-->
['bracket ']
Rear brake pipe grommet 1 

Rear seat belt bottom mtg reinf .  is ::-->
['seat  belt']
bkt.  is ::-->
['bracket ']
Rear seat squab holding crossmember .  is ::-->
['rear seat']
Bumper mounting bkt .  is ::-->
['bracket ']
Wiring harness holding bkt .  is ::-->
['bracket ']
Mudliner bkt .  is ::-->
['bracket ']
C pillar reinf on wheel house outer .  is ::-->
['c pillar']
Rear seat belt anchor mtg.  is ::-->
['seat  belt']
Back rest seat lock reinf.  is ::-->
['seat ']
Reinf latch outer .  is ::-->
['latch ']
Dashboard mtg.  is ::-->
['dashboard ']
Seat belt mtg .  is ::-->
['seat  belt']
Cant rail B pillar reinf.  is ::-->
['b pillar']
A pillar lower outer reinf .  is ::-->
['a pillar']
Door mounting hinge reinf .  is ::-->
['door ']
B pillar inner bottom .  is ::-->
['b pillar']
2nd Roof bow mtg bkt .  is ::-->
['bracket ']
Tail lamp mounting bkt .  is ::-->
['bracket ']
Front door inner panel .  is ::-->
['panel ']
Glass Run channel hinge side .  is ::-->
['glass ']
Rear door outer panel .  is ::-->
['panel ']
S

LONG MEMBER REAR MODULE LH .  is ::-->
['long member ']
SUPPORT PLATE LH .  is ::-->
['plate ']
BASE PLATE .  is ::-->
['plate ']
BRACKET HEIGHT SENSOR LINK MTG .  is ::-->
['bracket ']
BRACKET AIR BELLOW MTG FRONT-LH .  is ::-->
['bracket ']
OUTRIGGER ASSY FRONT-LH .  is ::-->
['outrigger ']
OUTRIGGER ASSY FRONT-LH .  is ::-->
['outrigger ']
OUTRIGGER ASSY FUEL TANK .  is ::-->
['fuel tank']
CLAMP ASSY EGP MTG .  is ::-->
['clamp ']
ASSY FOR PARALLEL ROD MGT .  is ::-->
['frame assy ']
BRACKET ( NO LOSS TANK MOUNTING ) .  is ::-->
['bracket ']
BRACKET ASSY CROSS MEMBER MTG FRONT-RH .  is ::-->
['bracket ']
HOSE HUMP TURBOCHARGER ADAPTER TO PIPE .  is ::-->
['hose ']
PUMP ASSY UREA DOSE .  is ::-->
['pump ']
VENT TUBE ASSY AUX WATER TANK TO ENGINE .  is ::-->
['hose ']
COVER ECU PROTECTION .  is ::-->
['cover ']
PARABOLIC LEAF SPRING ASSY FRONT .  is ::-->
['parabolic leaf spring']
PARABOLIC LEAF SPRING ASSY FRONT .  is ::-->
['parabolic leaf spring']
BRACKET EXHAUST PIPE MTG .  is ::-

...

PANEL ASSY FRONT .  is ::-->
['panel ']
BRAKE PEDAL ASSY WITH BUSH .  is ::-->
['brake  pedal']
HOSE ASSY CMC TO RESERVOIR .  is ::-->
['hose ']
BUNDY TUBE ASSY LONG MEMBER .  is ::-->
['long member ']
BRACKET .  is ::-->
['bracket ']
LONGMEMBER;RH .  is ::-->
['long member ']
"LONGMEMBER,LH" .  is ::-->
['long member ']
"LONGMEMBER,LH" .  is ::-->
['long member ']
"LONGMEMBER,RH" .  is ::-->
['long member ']
WIRING HARNESS TAIL .  is ::-->
['tail wiring harness']
TUBE SUCTION ASSY .  is ::-->
['tube assy ']
BRACKET... .  is ::-->
['bracket ']
Balancer Shaft Assy with Pin .  is ::-->
['shaft ']
FUEL RETURN PIPE ASSY .  is ::-->
['return hose']
FUEL SUCTION PIPE ASSY .  is ::-->
['suction  hose']
FUEL RETURN PIPE ASSY .  is ::-->
['return hose']
FUEL RETURN PIPE ASSY .  is ::-->
['return hose']
FUEL SUCTION PIPE ASSY .  is ::-->
['suction  hose']
BRACKET ASSY .  is ::-->
['bracket ']
"HOSE,VACUUM PUMP TO NRV" .  is ::-->
['heater  hose']
FUEL FILTER ASSY WITH BRACKET .  is ::-->
['fuel 

CROSS MEMBER ASSY REAR FLOOR .  is ::-->
['cross member ']
Bearing Camshaft Lower .  is ::-->
['bearing ']
METAL PIPE JOINT TO JOINT .  is ::-->
['metal pipe']
PIPE BUNCH ASSY .  is ::-->
['nylon bunch']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
GUARD PROPELLER SHAFT .  is ::-->
['propeller shaft guard']
"WIRING HARNESS,MAIN" .  is ::-->
['floor wiring harness']
"BRACKET,CENTER BRG MTG" .  is ::-->
['bracket ']
BRACKET ASSY;STG RESERVOIR MTG .  is ::-->
['bracket ']
"COCKPIT, TIAGO MCE XZ+ (NEW)" .  is ::-->
['cockpit wiring harness']
CYLINDER HEAD" .  is ::-->
['head  cylinder']
WHEEL RIM ASSY WITH VALVE 552940100106 .  is ::-->
['wheel rim ']
BRACKET .  is ::-->
['bracket ']
ANTI COMPOUNDING RELAY VALVE .  is ::-->
['relay  valve']
RELAY VALVE & MODULATOR VALVE ASSY .  is ::-->
['relay  valve']
NYLON PIPE BUNCH ASSY .  is ::-->
['nyl

Cyl Pin (Range fork) .  is ::-->
['pin ']
Deep Groove Ball Bearing .  is ::-->
['bearing ']
Top plate assy GSL bracket .  is ::-->
['bracket ']
BKT ASSY EMS ECU MTG .  is ::-->
['bracket ']
HEATER HOSE OUTLET .  is ::-->
['heater  hose']
POWER STEERING GEAR ASSY .  is ::-->
['gear ']
BRAKE DRUM (140mm) .  is ::-->
['drum   brake']
"WIRING HARNESS,EMS" .  is ::-->
['floor wiring harness']
1st/5th Gr Counter Shaft .  is ::-->
['counter  shaft']
BRACKET ASSY WITH FASTENER .  is ::-->
['bracket ']
"HOSE,CLAMP" .  is ::-->
['heater  hose']
HOSE EVRV CIRCUIT RESERVIOR .  is ::-->
['hose ']
BRACKET ASSY AIR FILTER MTG .  is ::-->
['bracket ']
"HOSE,THERMOSTAT COVER TO RADIATOR" .  is ::-->
['heater  hose']
"FOG LAMP ASSY,FRONT-LH" .  is ::-->
['fog  lamp']
TAIL LAMP ASSY BODY SIDE OUTER... .  is ::-->
['tail  lamp']
Head lamp RH - high .  is ::-->
['head  lamp']
"TAIL LAMP ASSY ,BODY SIDE,OUTE..." .  is ::-->
['tail  lamp']
HOSE ASSY SUCTION .  is ::-->
['suction  hose']
"W/H, FRONT TIAGO MCE

FRAME ASSY WITH TOP COAT .  is ::-->
['frame assy ']
"BRACKET,AGB SHIFT LEVER MTG ASSY" .  is ::-->
['bracket ']
FRAME ASSY WITH FITMENT .  is ::-->
['frame assy ']
"LONG MEMBER,RH" .  is ::-->
['long member ']
"WIRING HARNESS,SIDE MARKER LAMP" .  is ::-->
['floor wiring harness']
BRACKET .  is ::-->
['bracket ']
"BRACKET ASSY,EXHAUST PIPE" .  is ::-->
['bracket ']
"EXHAUST PIPE ASSY,FIRST" .  is ::-->
['pipe assy ']
BRACKET ASSY .  is ::-->
['bracket ']
"BRACKET ASSY, EXHAUST PIPE SUPPORT" .  is ::-->
['bracket ']
BRACKET ASSY .  is ::-->
['bracket ']
LOAD BODY WITH MOUNTINGS .  is ::-->
['load body ']
BRACKET .  is ::-->
['bracket ']
VACUUM HOSE ASSY .  is ::-->
['vacuum  hose']
BRAKE PEDAL ASSY WITH CONTROL MTG BKT .  is ::-->
['brake  pedal']
"CARPET ASSY,FLOOR" .  is ::-->
['carpet ']
BOOSTER ASSY WITH TMC & RESERVOIR .  is ::-->
['booster ']
Frame Assy .  is ::-->
['frame assy ']
Frame Assy .  is ::-->
['frame assy ']
Frame Assy .  is ::-->
['frame assy ']
Frame Assy .  is ::-->


OIL SEAL MAIN SHAFT 38X48X8 (M/S CR) .  is ::-->
['oil seal ']
ACCELERATOR PEDAL MODULE .  is ::-->
['accelerator  pedal']
FUEL HOSE .  is ::-->
['fuel return hose']
OIL PUMP ASSY .  is ::-->
['oil pump']
INNER COVER ASSY;ORVM;LH .  is ::-->
['cover ']
BUSH;CAB MTG;REAR .  is ::-->
['sintered bush']
BRACKET ASSY;MTG .  is ::-->
['bracket ']
FUEL RETURN PIPE ASSY .  is ::-->
['return hose']
FUEL RETURN PIPE ASSY .  is ::-->
['return hose']
FUEL SUCTION PIPE ASSY .  is ::-->
['suction  hose']
SPACER .  is ::-->
['rubber spacer']
HP TUBE ASSY (CYLINDER VALVE TO UNDERBODY-JOINT) - Tiago .  is ::-->
['hp tube ']
CROSS MEMBER ASSY .  is ::-->
['cross member ']
Frame .  is ::-->
['frame assy ']
FRAME ASSY WITH FITMENT .  is ::-->
['frame assy ']
SUPPORT BRACKET ASSY .  is ::-->
['bracket ']
"Bkt Assy, Battery Cut-off switch mtg" .  is ::-->
['bracket ']
"CABLE, HV S-BOX +VE TO BATT PACK-6 +VE" .  is ::-->
['cable tie ']
"CABLE, HV BATT P-3 -VE TO BATT P-2 +VE" .  is ::-->
['cable tie ']
"CABL

In [21]:
final_doc=[]
init_size = len(clean_doc)
for line in clean_doc:
    
    line1 = line.lower()
    string = line1.partition(",")[0].partition(";")[0].partition("(")[0].partition("[")[0].partition('for')[0].partition('with')[0].partition('w/o')[0].partition('offer')[0].partition('\bw\b')[0]

    for x in dictionary:
        
        if (secondsearch(replacer(string.lower()),dictionary[x].lower(), line) == 1):
            print(line, " is ::-->")
            break
    if(len(final_doc)!=0):
        print(final_doc)
    final_doc = [] 

SUPPORT BRACKET (ENGINE) .  is ::-->
['bracket ']
"Holder Parking Sensor, Rear RH" .  is ::-->
['parking  sensor']
TOP PANEL ASSY (Buy off Level) .  is ::-->
['panel ']
"WHEEL ARCH LINER ASSY,REAR-RH" .  is ::-->
['wheel arch liner']
ASSY A/C COMPRESSOR TCCI Make .  is ::-->
['compressor ']
SUCTION PIPE ASSY TANK .  is ::-->
['suction  hose']
BRACKET HYD RESERVOIR SUPPORT .  is ::-->
['bracket ']
WIRING HARNESS REAR CHASSIS 29WB .  is ::-->
['wiring harness ']
BRACKET OIL COOLER ASSY .  is ::-->
['bracket ']
COUNTER SHAFT ASSY .  is ::-->
['counter  shaft']
"COVER ASSY WITH BUSH,REAR" .  is ::-->
['cover ']
THIRD SPEED GEAR - TCIC .  is ::-->
['gear ']
GRANIT_BLACK-DRIVER SEAT ASSY .  is ::-->
['driver seat']
"FLOOR,MIDDLE" .  is ::-->
['floor wiring harness']
"CLRPRT_ASY_1-DOOR TRIM ASSY,FRT DR M" .  is ::-->
['door trim']
DOOR OUTRIGGER ASSY .  is ::-->
['door ']
KIT-PROPELLER SHAFT .  is ::-->
['propeller  shaft']
PLATE RADIATOR MTG BRACKET .  is ::-->
['bracket ']
WIRING HARNESS ES

Dashboard RH cover(Brake oil reservoir cover) .  is ::-->
['cover ']
Body shell sealing grommet 1(Front panel) .  is ::-->
['grommet ']
A pillar upper trim mounting screw button LH .  is ::-->
['mounting screw ']
Front door lock / unlock operating link from inner opening lever clamp .  is ::-->
['clamp ']
Wiper motor mounting nut to link (II) .  is ::-->
['mounting nut ']
Front windshield washer sprinkler pipe holding clamp 1 .  is ::-->
['clamp ']
Front bumper locater bracket mounting screw .  is ::-->
['mounting screw ']
Dead Pedal .  is ::-->
['dead pedal  pedal']
Blow by hose (From crank case ventilation to Oil sump) .  is ::-->
['hose ']
Engine balancer shaft rear bearing .  is ::-->
['bearing ']
Air filter snorkel bellow from snorkel air intake pipe 2 to air intake pipe 3 .  is ::-->
['air intake pipe']
Vacuum hose 1 to EGR .  is ::-->
['vacuum  hose']
Reverse gear shifter shaft .  is ::-->
['shifter  shaft']
Third gear synchronizer cone .  is ::-->
['gear ']
Front RH and LH leaf

Engine access LH and RH cover holding Clamp .  is ::-->
['clamp ']
Oil sump mounting nut .  is ::-->
['mounting nut ']
Blow by hose clip 1 .  is ::-->
['hose ']
Exhaust valve rocker arm lifter lock .  is ::-->
['valve ']
Fuel non return valve bracket mounting bolt .  is ::-->
['mounting bolt']
Air filter mounting bush .  is ::-->
['bush ']
Intake manifold seal .  is ::-->
['seal ']
EGR pipe from cylinder head (internally from exhaust) to EGR body mounting bolt .  is ::-->
['egr pipe']
Turbo charger lubricating pipe bracket mounting nut .  is ::-->
['mounting nut ']
Vacuum switch bracket mounting bolt .  is ::-->
['mounting bolt']
Engine mount bracket mounting bolt (2) on engine (front LH) .  is ::-->
['mounting bolt']
Shifter sleeve detainer of fifth gear Synchronizer unit on input shaft .  is ::-->
['sleeve ']
First and second gear shifter link mounting bolt .  is ::-->
['mounting bolt']
Lay shaft bearing circlip (Front end) .  is ::-->
['bearing ']
Speedometer gear circlip on output 

HEX.BOLT (M12X1.5X35) .  is ::-->
['bolt ']
BEAM ASSY RUPD .  is ::-->
['beam ']
SWITCH FACIA DUMMY LH .  is ::-->
['switch ']
BRACKET AND OAT SENSOR ASSY .  is ::-->
['bracket ']
HEX BOLT (M12X1.5X150) .  is ::-->
['bolt ']
ASSY.BRACKET .  is ::-->
['bracket ']
ASSY PIPE FOR CLAMP FOR STG HOSE .  is ::-->
['pipe assy ']
TEMPORARY NUMBER PLATE VINYL STICKER-LKN .  is ::-->
['plate ']
PIPE RETARDER TO ENGINE .  is ::-->
['pipe ']
BRACKET ASSY RADIATOR GUARD LH .  is ::-->
['bracket ']
BRACKET ASSY RADIATOR GUARD LH .  is ::-->
['bracket ']
"INSULATION,REAR WHEEL ARCH LINER,REAR" .  is ::-->
['insulation ']
ASSY.PROTECTIVE COVER .  is ::-->
['cover ']
PIPE METAL ASSY .  is ::-->
['metal pipe']
Drum brake assy LH 410X220 .  is ::-->
['drum   brake']
PUMP ASSY UREA DOSE .  is ::-->
['pump ']
AIR SUPPLY PIPE ASSY AIR COMPRESSOR .  is ::-->
['pipe assy ']
BRACKET TELEMATICS UNIT MTG .  is ::-->
['bracket ']
BKT LID DIRTY STORAGE BOX UPPER RH M0 .  is ::-->
['bracket ']
DOOR TRIM LH .  is ::-

Fr Strut Damper Assy - LH - 543832301605 .  is ::-->
['strut ']
Gasket .  is ::-->
['gasket ']
"DOOR MEMBRANE,LH" .  is ::-->
['door ']
"DOOR MEMBRANE,RH" .  is ::-->
['door ']
"SUPPORT PLATE,ENGINE MTG" .  is ::-->
['plate ']
"BRACKET ASSY,GEAR BOX MTG,RH" .  is ::-->
['bracket ']
Stub Axle ABS -RH .  is ::-->
['stub axle']
"BRACKET,POWER CYL MTG" .  is ::-->
['bracket ']
BRACKET CLAMP MTG STG PIPE .  is ::-->
['bracket ']
RETURN LINE ASSY RESERVOIR END .  is ::-->
['return hose']
CLUTCH RELEASE BEARING-OFFER DRG .  is ::-->
['bearing ']
BRACKET CABLE SHIFT .  is ::-->
['bracket ']
UREA TEMPERATURE LEVEL & QUALITY SENSOR .  is ::-->
['urea  sensor']
BRACKETCLEAN SIDE PIPE SUPPORT .  is ::-->
['pipe ']
HOSE REDUCER .  is ::-->
['hose ']
PIPE AIR FILTER TO PRE FILTER .  is ::-->
['pipe ']
HOSE CLAMP (FGI) .  is ::-->
['clamp ']
HOSE RADIATOR INTAKE .  is ::-->
['hose ']
CYLINDER HEAD .  is ::-->
['head  cylinder']
SNORKEL ASSY AIR INTAKE .  is ::-->
['snorkel ']
BRACKET ASSY-BOTTOM AIR 

...

"BRACKET ASSY,RADIATOR MTG,BOTTOM" .  is ::-->
['bracket ']
"DRUM BRAKE ASSY WITH DC,FRONT-RH" .  is ::-->
['drum   brake']
WATER PUMP ASSY .  is ::-->
['water pump']
BUSH .  is ::-->
['sintered bush']
REAR AXLE BEAM ASSY .  is ::-->
['rear axle  beam']
NYLON PIPE ASSY .  is ::-->
['nylon pipe']
DRUM BRAKE ASSY WITH DC FRONT-LH .  is ::-->
['drum   brake']
PIPE BUNCH ASSY CHASSIS .  is ::-->
['nylon bunch']
"CABLE,HV AUX INV TO AUX DRIVE MOTOR" .  is ::-->
['cable tie ']
"CABLE,HV MOTOR INVERTER-1 TO MOTOR-C2" .  is ::-->
['cable tie ']
ASSY MTG BKT HEATER PIPING .  is ::-->
['bracket ']
PIPE ASSY .  is ::-->
['pipe assy ']
TUBE .  is ::-->
['hp tube ']
VENT PIPE ASSY .  is ::-->
['pipe assy ']
VENT TUBE ASSY .  is ::-->
['hose ']
DRUM BRAKE ASSY WITH DC REAR-LH .  is ::-->
['drum   brake']
DRUM BRAKE ASSY WITH DC REAR-LH .  is ::-->
['drum   brake']
DRUM BRAKE ASSY WITH DC REAR-LH .  is ::-->
['drum   brake']
DRUM BRAKE ASSY WITH DC REAR-LH .  is ::-->
['drum   brake']
DRUM BRAKE ASSY

SHROUD FOR VISCOUS 430mm FAN .  is ::-->
['fan shroud']
"HOSE ASSY,TURBOCHARGER TO INTERCOOLER" .  is ::-->
['hose ']
AIR SUPPLY PIPE ASSY .  is ::-->
['pipe assy ']
PIPE .  is ::-->
['air intake pipe']
"HOSE, TUBE TO T CONNECTOR" .  is ::-->
['heater  hose']
TUBE .  is ::-->
['hp tube ']
AIR SUPPLY PIPE A/F TO AIR COMPRESSOR .  is ::-->
['pipe ']
DIPSTICK ASSY .  is ::-->
['dipstick ']
"COVER,FRONT" .  is ::-->
['engine cover']
BREATHER HOSE .  is ::-->
['breather  hose']
TUBE (UREA INJ TO CONNECTOR) .  is ::-->
['hp tube ']
TUBE OUTLET (TUBE END TO T-CONNECTOR ASSY) .  is ::-->
['tube ']
BRACKET (FOR FIT.ON R.A.109 ) .  is ::-->
['bracket ']
GUSSET CROSS MEMBER BOTTOM-LH .  is ::-->
['cross member ']
BRACKET ASSY .  is ::-->
['bracket ']
AIR TANK ASSY WITH MTG BKT .  is ::-->
['air tank']
RETURN LINE DM TO TANK .  is ::-->
['return hose']
RETURN FUEL PIPE ASSY ENG TO FUEL TANK .  is ::-->
['pipe assy ']
PRESSURE LINE SM TO DM .  is ::-->
['hose ']
BRACKET ASSY RESERVOIR MTG .  is ::-

WIRING HARNESS;ABS PIGTAIL .  is ::-->
['floor wiring harness']
"L-BRACKET,EXHAUST PIPE MTG" .  is ::-->
['bracket ']
POLY V-BELT-OFFER DRG .  is ::-->
['poly  belt']
PANEL ROOF CENTER FRONT LOW .  is ::-->
['panel ']
PIPE BUNCH ASSY .  is ::-->
['nylon bunch']
L-BRACKET .  is ::-->
['bracket ']
Tube Coolant supply intermediate .  is ::-->
['tube ']
BRACKET .  is ::-->
['bracket ']
BATTERY TRAY ASSY .  is ::-->
['tray ']
"B-PILLAR ASSY,INNER-RH" .  is ::-->
['b pillar']
"PANEL ASSY,C-RAIL,INNER,REAR-RH" .  is ::-->
['panel ']
PROPELLER SHAFT ASSY DRIVE .  is ::-->
['drive  shaft']
PIPE BUNCH ASSY .  is ::-->
['nylon bunch']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
INSULATION .  is ::-->
['insulation ']
"WIRING HARNESS,SCR UNIT" .  is ::-->
['floor wiring harness']
"BRACKET,CENTER BRG MTG" .  is ::-->
['bracket ']
PIPE ASSY .  is ::-->
['pipe assy ']
CYLINDER HEAD" .  is ::

"REAR AXLE BEAM ASSY,REAR-FWD" .  is ::-->
['rear axle  beam']
REAR AXLE BEAM ASSY .  is ::-->
['rear axle  beam']
Battery Cable +ve .  is ::-->
['cable ']
"LONG MEMBER, LH" .  is ::-->
['long member ']
"BRACKET,CENTER BRG MTG" .  is ::-->
['bracket ']
"Guide Plate, Rear Spring" .  is ::-->
['plate ']
RUBBER BOLSTER MOUNTING BRACKET LH .  is ::-->
['bracket ']
SPACER .  is ::-->
['rubber spacer']
COMBI SWITCH ASSY .  is ::-->
['combi  switch']
"PARKING BRAKE CABLE ASSY,FRONT" .  is ::-->
['brake cable']
"PRIMARY BODY SEAL,TAILGATE" .  is ::-->
['primary seal ']
Dust Seal .  is ::-->
['seal ']
METAL PIPE ASSY .  is ::-->
['metal pipe']
"LONG MEMBER,LH" .  is ::-->
['long member ']
LEAF SPRING ASSY REAR .  is ::-->
['leaf spring ']
PIPE BUNCH ASSY .  is ::-->
['nylon bunch']
Return line .  is ::-->
['return hose']
"WIRING HARNESS,ENGINE" .  is ::-->
['floor wiring harness']
"CORNER PANEL ASSY WITH POST,OUTER-LH" .  is ::-->
['panel ']
Gear shift lever assy .  is ::-->
['gear shift  lever

"EXHAUST PIPE ASSY,SECOND" .  is ::-->
['pipe assy ']
EXHAUST PIPE ASSY .  is ::-->
['pipe assy ']
BRACKET ASSY BATTERY +VE CABLE MTG .  is ::-->
['bracket ']
BRAKE DRUM .  is ::-->
['drum   brake']
SPINDLE ASSY REAR SUSP TWIST BEAM - MY15 .  is ::-->
['spindle ']


In [10]:
closest_match[2]

215